In [1]:
import numpy as np
import pandas as pd
import scipy.integrate
import scipy.interpolate
import scipy.optimize

import bokeh.io
bokeh.io.output_notebook()
import panel as pn
pn.extension()

def style(p, autohide=False):
    p.title.text_font="Helvetica"
    p.title.text_font_size="16px"
    p.title.align="center"
    p.xaxis.axis_label_text_font="Helvetica"
    p.yaxis.axis_label_text_font="Helvetica"
    
    p.xaxis.axis_label_text_font_size="13px"
    p.yaxis.axis_label_text_font_size="13px"
    p.background_fill_alpha = 0
    if autohide: p.toolbar.autohide=True
    return p

Loading BokehJS ...

In [2]:
dark_green = "#31a354"
light_green = "#74c476"
dark_orange = "#e6550d"
light_orange = "#fd8d3c"

# double occupancy

In [3]:
def derivs_double(AR, t, alpha, beta, gamma, kappa, n):
    A, R = AR
    f = (kappa*A)**n / ((1 + (kappa*A)**n)*(1 + R**n))
    
    deriv_A = alpha + beta * f - gamma*A
    deriv_R = alpha + beta * f - R
    return np.array([deriv_A, deriv_R])

In [4]:
n = 4
t = np.linspace(0, 20, 500)

In [5]:
log_alpha_slider = pn.widgets.FloatSlider(name="log α", start=-1, end=2, step=0.1, value=-0.7, width=150)
log_beta_slider = pn.widgets.FloatSlider(name="log β", start=0, end=4, step=0.1, value=2, width=150)
log_gamma_slider = pn.widgets.FloatSlider(name="log γ", start=0, end=4, step=0.1, value=1.9, width=150)
log_kappa_slider = pn.widgets.FloatSlider(name="log κ", start=0, end=4, step=0.1, value=1.5, width=150)
n_slider = pn.widgets.IntSlider(name="n", start=1, end=10, value=3, width=100)

@pn.depends(log_alpha_slider.param.value, 
            log_beta_slider.param.value, 
            log_gamma_slider.param.value, 
            log_kappa_slider.param.value, 
            n_slider.param.value)
def AR_plotter(log_alpha, log_beta, log_gamma, log_kappa, n):
    alpha, beta = np.power(10.0, log_alpha), np.power(10.0, log_beta)
    gamma, kappa =  np.power(10.0, log_gamma), np.power(10.0, log_kappa)
    
    args = (alpha, beta, gamma, kappa, n)

    ARo = np.array([1, 1])
    _AR = scipy.integrate.odeint(derivs_double, ARo, t, args=args)
    A, R = _AR.T
    p = bokeh.plotting.figure(height=350, width=450,
                            title="time traces", x_axis_label="time", y_axis_label="[ ]")
    p.line(t, A, line_color=light_green, line_width=3)
    p.line(t, R, line_color=light_orange, line_width=3)

    return style(p)


widgets = pn.Row(pn.Column(log_alpha_slider, log_beta_slider), 
                 pn.Column(log_gamma_slider, log_kappa_slider), 
                 pn.Column(n_slider, align="center"), align="center")
pn.Column(AR_plotter, widgets)

Column
    [0] ParamFunction(function)
    [1] Row(align='center')
        [0] Column
            [0] FloatSlider(end=2, name='log α', start=-1, value=-0.7, width=150)
            [1] FloatSlider(end=4, name='log β', value=2, width=150)
        [1] Column
            [0] FloatSlider(end=4, name='log γ', value=1.9, width=150)
            [1] FloatSlider(end=4, name='log κ', value=1.5, width=150)
        [2] Column(align='center')
            [0] IntSlider(end=10, name='n', start=1, value=3, width=100)

oscillations only when log alpha = 0, or when alpha = 1 or less
but the regions in which it oscillates are very sparse

# OR logic

In [6]:
# function for integrations
def derivs_AND(AR, t, alpha, beta, gamma, kappa, n):
    A, R = AR
    f = (kappa*A)**n / (1 + (kappa*A)**n + R**n)
    
    deriv_A = alpha + beta * f - gamma*A
    deriv_R = alpha + beta * f - R
    return np.array([deriv_A, deriv_R])


def derivs_OR(AR, t, alpha, beta, gamma, kappa, n):
    A, R = AR
    f = (1+(kappa*A)**n) / (1 + (kappa*A)**n + R**n)
    
    deriv_A = alpha + beta * f - gamma*A
    deriv_R = alpha + beta * f - R
    return np.array([deriv_A, deriv_R])

In [7]:
log_alpha_slider = pn.widgets.FloatSlider(name="log α", start=0, end=4, step=0.1, value=0, width=150)
log_beta_slider = pn.widgets.FloatSlider(name="log β", start=0, end=4, step=0.1, value=2, width=150)
log_gamma_slider = pn.widgets.FloatSlider(name="log γ", start=0, end=4, step=0.1, value=1.5, width=150)
log_kappa_slider = pn.widgets.FloatSlider(name="log κ", start=0, end=4, step=0.1, value=1.5, width=150)
n_slider = pn.widgets.IntSlider(name="n", start=1, end=10, value=3, width=100)

@pn.depends(log_alpha_slider.param.value, log_beta_slider.param.value, 
            log_gamma_slider.param.value, log_kappa_slider.param.value, 
            n_slider.param.value)
def AR_plotter(log_alpha, log_beta, log_gamma, log_kappa, n):
    alpha, beta = np.power(10.0, log_alpha), np.power(10.0, log_beta)
    gamma, kappa =  np.power(10.0, log_gamma), np.power(10.0, log_kappa)
    
    args = (alpha, beta, gamma, kappa, n)

    ARo = np.array([0.1, 0.1])
    _AR = scipy.integrate.odeint(derivs_OR, ARo, t, args=args)
    A, R = _AR.T
    p = bokeh.plotting.figure(height=350, width=450,
                            title="time traces", x_axis_label="time", y_axis_label="[ ]")
    p.line(t, A, line_color=light_green, line_width=3)
    p.line(t, R, line_color=light_orange, line_width=3)
    
    _AR = scipy.integrate.odeint(derivs_AND, ARo, t, args=args)
    A, R = _AR.T
    q = bokeh.plotting.figure(height=350, width=450,
                            title="time traces", x_axis_label="time", y_axis_label="[ ]")
    q.line(t, A, line_color=light_green, line_width=3)
    q.line(t, R, line_color=light_orange, line_width=3)
    return pn.Row(style(p), style(q))


widgets = pn.Row(pn.Column(log_alpha_slider, log_beta_slider), 
                 pn.Column(log_gamma_slider, log_kappa_slider), 
                 pn.Column(n_slider, align="center"), align="center")
pn.Column(AR_plotter, widgets)

Column
    [0] ParamFunction(function)
    [1] Row(align='center')
        [0] Column
            [0] FloatSlider(end=4, name='log α', width=150)
            [1] FloatSlider(end=4, name='log β', value=2, width=150)
        [1] Column
            [0] FloatSlider(end=4, name='log γ', value=1.5, width=150)
            [1] FloatSlider(end=4, name='log κ', value=1.5, width=150)
        [2] Column(align='center')
            [0] IntSlider(end=10, name='n', start=1, value=3, width=100)

In [8]:
def fixed_point(alpha, beta, gamma, kappa, n, low=0, high=100):
    def _root_function(a):
        coeff1 = gamma*(gamma**n + kappa**n)
        coeff2 = (kappa**n)*(alpha + beta) + alpha*gamma**n
        coeff3 = gamma
        coeff4 = alpha + beta
        return coeff1*a**(n+1) - coeff2*a**n + coeff3*a - coeff4
    return scipy.optimize.brentq(_root_function, alpha/gamma, (alpha+beta)/gamma)

def retrieve_eigens(A, alpha, beta, gamma, kappa, n):
    R = gamma * A
    denom = (1+ (A*kappa)**n + R**n)**2
    one = beta * n * (A*kappa)**n * (R**n) / A / denom - gamma
    two = - beta * n * (1+(A*kappa)**n) * R**(n-1) / denom
    three = beta * n * (A*kappa)**n * (R**n) / A / denom
    four = - beta * n * (1+(A*kappa)**n) * R**(n-1) / denom - 1
    J = np.array([[one, two], [three, four]])
    
    return np.linalg.eigvals(J)

In [9]:
grid = 200
n = 4

In [10]:
alpha, beta = 5, 80
k, g = np.linspace(0.1, 50, grid), np.linspace(0.1, 50, grid)
kk, gg = np.meshgrid(k, g)

fps = np.empty([grid, grid])
for i in range(grid):
    for j in range(grid):
        kappa, gamma = kk[i, j], gg[i, j]
        fps[i, j] = fixed_point(alpha, beta, gamma, kappa, n, high=1e4)
        
orange = "#fd8d3c" 
green = "#74c476"
purple = "#966DA1"
blue = "#5779A3"

colors = np.empty([grid, grid], dtype=object)
for i in range(grid):
    for j in range(grid):
        A_fp = fps[i, j]
        gamma, kappa = gg[i, j], kk[i, j]
        eig1, eig2 = retrieve_eigens(A_fp, alpha, beta, gamma, kappa, n)
        if (eig1 < 0 and eig2 < 0): colors[i, j] = orange
        if (eig1 > 0 and eig2 > 0): colors[i, j] = purple
        if (eig1 > 0 and eig2 < 0) or (eig1 < 0 and eig2 > 0): colors[i, j] = green
            
_df_kappas, _df_gammas, _df_colors = [], [], []

for i in range(grid):
    for j in range(grid):
        kappa, gamma = kk[i, j], gg[i, j]
        color = colors[i, j]
        _df_kappas.append(kappa)
        _df_gammas.append(gamma)
        _df_colors.append(color) 
df = pd.DataFrame({'kappa':_df_kappas, 'gamma':_df_gammas, 'color':_df_colors})

p = bokeh.plotting.figure(title="α = 5, β = 80", x_axis_label="kappa", y_axis_label="gamma", 
                          x_range=(0, 50), y_range=(0, 50), height=400, width=400)
p.circle(source=df, x='kappa', y='gamma', color='color')

GlyphRenderer(id='1567', ...)

In [11]:
gamma, kappa = 10, 10
a, b = np.linspace(0.1, 30, grid), np.linspace(0.1, 100, grid)
aa, bb = np.meshgrid(a, b)

fps = np.empty([grid, grid])
for i in range(grid):
    for j in range(grid):
        alpha, beta = aa[i, j], bb[i, j]
        fps[i, j] = fixed_point(alpha, beta, gamma, kappa, n, low=alpha/gamma, high=(alpha+beta)/gamma)
        
        
        
colors = np.empty([grid, grid], dtype=object)
for i in range(grid):
    for j in range(grid):
        A_fp = fps[i, j]
        alpha, beta = aa[i, j], bb[i, j]
        eig1, eig2 = retrieve_eigens(A_fp, alpha, beta, gamma, kappa, n)
        if (eig1 < 0 and eig2 < 0): colors[i, j] = orange
        if (eig1 > 0 and eig2 > 0): colors[i, j] = purple
        if (eig1 > 0 and eig2 < 0) or (eig1 < 0 and eig2 > 0): colors[i, j] = green
            
            
            
_df_alphas, _df_betas, _df_colors = [], [], []

for i in range(grid):
    for j in range(grid):
        alpha, beta = aa[i, j], bb[i, j]
        color = colors[i, j]
        _df_alphas.append(alpha)
        _df_betas.append(beta)
        _df_colors.append(color) 
df = pd.DataFrame({'alpha':_df_alphas, 'beta':_df_betas, 'color':_df_colors})

q = bokeh.plotting.figure(title="γ = 10, κ = 10", x_axis_label="alpha", y_axis_label="beta", 
                          x_range=(0.1, 30), y_range=(0.1, 100), height=400, width=400)
q.circle(source=df, x='alpha',y='beta', color='color')



bokeh.io.show(bokeh.layouts.layout([[style(p), style(q)]]))